# Stock Dataset Process

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import optuna

Original dataset from Kaggle: https://www.kaggle.com/datasets/andrewmvd/sp-500-stocks/data

In [2]:
comp0=pd.read_csv('../input/sp-500-stocks/sp500_companies.csv')
display(comp0[0:11])

,Exchange,Symbol,Shortname,Longname,Sector,Industry,Currentprice,Marketcap,Ebitda,Revenuegrowth,City,State,Country,Fulltimeemployees,Longbusinesssummary,Weight
0,NMS,MSFT,Microsoft Corporation,Microsoft Corporation,Technology,Software - Infrastructure,400.96,2979309027328,1.184270e+11,0.176,Redmond,WA,United States,221000.0,Microsoft Corporation develops and supports so...,0.063768
1,NMS,AAPL,Apple Inc.,Apple Inc.,Technology,Consumer Electronics,165.84,2560884473856,1.301090e+11,0.021,Cupertino,CA,United States,161000.0,"Apple Inc. designs, manufactures, and markets ...",0.054812
2,NMS,NVDA,NVIDIA Corporation,NVIDIA Corporation,Technology,Semiconductors,795.18,1987950018560,3.448000e+10,2.653,Santa Clara,CA,United States,29600.0,NVIDIA Corporation provides graphics and compu...,0.042549
3,NMS,GOOG,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,157.95,1952830586880,1.001720e+11,0.135,Mountain View,CA,United States,182502.0,Alphabet Inc. offers various products and plat...,0.041798
4,NMS,GOOGL,Alphabet Inc.,Alphabet Inc.,Communication Services,Internet Content & Information,156.28,1952218480640,1.001720e+11,0.135,Mountain View,CA,United States,182502.0,Alphabet Inc. offers various products and plat...,0.041785
5,NMS,AMZN,"Amazon.com, Inc.","Amazon.com, Inc.",Consumer Cyclical,Internet Retail,177.23,1843670548480,8.551500e+10,0.139,Seattle,WA,United States,1525000.0,"Amazon.com, Inc. engages in the retail sale of...",0.039461
6,NMS,META,"Meta Platforms, Inc.","Meta Platforms, Inc.",Communication Services,Internet Content & Information,481.73,1221922521088,6.138100e+10,0.247,Menlo Park,CA,United States,67317.0,"Meta Platforms, Inc. engages in the developmen...",0.026154
7,NYQ,BRK-B,Berkshire Hathaway Inc. New,Berkshire Hathaway Inc.,Financial Services,Insurance - Diversified,408.78,883598426112,1.356820e+11,0.196,Omaha,NE,United States,396500.0,"Berkshire Hathaway Inc., through its subsidiar...",0.018912
8,NYQ,LLY,Eli Lilly and Company,Eli Lilly and Company,Healthcare,Drug Manufacturers - General,731.33,695323721728,1.231460e+10,0.281,Indianapolis,IN,United States,43000.0,"Eli Lilly and Company discovers, develops, and...",0.014882
9,NMS,AVGO,Broadcom Inc.,Broadcom Inc.,Technology,Semiconductors,1224.46,567440441344,2.040400e+10,0.164,Palo Alto,CA,United States,20000.0,"Broadcom Inc. designs, develops, and supplies ...",0.012145


In [3]:
comp1=comp0.sort_values('Weight',ascending=False)
names=comp1.loc[:,'Symbol'].tolist()
print(len(names))

503


In [4]:
stock0=pd.read_csv('../input/sp-500-stocks/sp500_stocks.csv')
stock0=stock0.dropna()
display(stock0)

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
0,2010-01-04,MMM,40.835056,69.414719,69.774246,69.122070,69.473244,3640265.0
1,2010-01-05,MMM,40.579273,68.979935,69.590302,68.311035,69.230766,3405012.0
2,2010-01-06,MMM,41.154747,69.958191,70.735786,69.824417,70.133781,6301126.0
3,2010-01-07,MMM,41.184269,70.008362,70.033447,68.662209,69.665550,5346240.0
4,2010-01-08,MMM,41.474468,70.501671,70.501671,69.648827,69.974915,4073337.0
...,...,...,...,...,...,...,...,...
1810292,2024-04-16,ZTS,152.584930,153.020004,154.399994,148.660004,150.199997,4967400.0
1810293,2024-04-17,ZTS,151.508011,151.940002,153.179993,151.240005,153.009995,3270500.0
1810294,2024-04-18,ZTS,153.110001,153.110001,153.669998,151.169998,151.630005,4772800.0
1810295,2024-04-19,ZTS,146.500000,146.500000,151.539993,145.800003,151.500000,6162800.0


In [5]:
stock0['Year'] = pd.to_datetime(stock0['Date']).dt.year
stock0['Close'] = stock0['Close'].to_frame()
# Calculate Exponential Moving Average (EWMA) with a span of 30
stock0['EWMA30'] = stock0['Close'].ewm(span=30).mean()

## Stock Dataset and Growth Rate Dataset

In [6]:
annual = pd.DataFrame(columns=['Symbol','Price Average','Volume Average','Price Annual Average Growth Rate','Volume Annual Average Growth Rate'])
for i in range(len(names)):
    data = stock0[stock0['Symbol']==names[i]].reset_index(drop=True)[['Symbol','Year','EWMA30','Volume']]
    data = data[data["Year"]>=2015]
    data = data.groupby(["Symbol","Year"])["EWMA30","Volume"].mean().reset_index()
    data['ewma30_pct'] = (data['EWMA30'].pct_change())*100
    data['volume_pct'] = (data['Volume'].pct_change())*100
    mean_price = round(data['EWMA30'].mean(),4)
    mean_volume = round(data["Volume"].mean(),4)
    mean_price_grow = round(data['ewma30_pct'].mean(),4)
    mean_volume_grow = round(data['volume_pct'].mean(),4)
    annual.loc[len(annual.index)] = [names[i], mean_price, mean_volume,mean_price_grow, mean_volume_grow]
    annual.reset_index(drop=True)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """


In [7]:
annual["Name"] = annual['Symbol'].map(comp0.set_index('Symbol')['Longname'])
annual["Sector"] = annual['Symbol'].map(comp0.set_index('Symbol')['Sector'])
annual["Marketcap"] = annual['Symbol'].map(comp0.set_index('Symbol')['Marketcap'])
annual['Rank'] = annual['Marketcap'].rank(ascending = 0)
annual['Marketcap %'] = (annual['Symbol'].map(comp0.set_index('Symbol')['Weight']))*100
annual.sort_values('Marketcap',ascending=False).reset_index()
annual.to_csv('stock.csv', index=False)

## Top 20 Dataset

In [8]:
df = stock0[stock0['Year']>=2015].reset_index()
df["Date"] = pd.to_datetime(df['Date'])
df["Name"] = df['Symbol'].map(comp0.set_index('Symbol')['Longname'])
df['Rank'] = df['Symbol'].map(annual.set_index('Symbol')['Rank'])
df = df[df["Rank"]<21].reset_index()
df = df.loc[:,["Date","Year","Symbol","Close","Name","Rank"]]
df.to_csv('annual.csv', index=False)

In [9]:
df

,Date,Year,Symbol,Close,Name,Rank
0,2015-01-02,2015,GOOGL,26.477501,Alphabet Inc.,5.0
1,2015-01-05,2015,GOOGL,25.973000,Alphabet Inc.,5.0
2,2015-01-06,2015,GOOGL,25.332001,Alphabet Inc.,5.0
3,2015-01-07,2015,GOOGL,25.257500,Alphabet Inc.,5.0
4,2015-01-08,2015,GOOGL,25.345501,Alphabet Inc.,5.0
...,...,...,...,...,...,...
46815,2024-04-16,2024,WMT,59.840000,Walmart Inc.,13.0
46816,2024-04-17,2024,WMT,59.650002,Walmart Inc.,13.0
46817,2024-04-18,2024,WMT,59.259998,Walmart Inc.,13.0
46818,2024-04-19,2024,WMT,59.529999,Walmart Inc.,13.0


## S&P 500 Dataset

In [10]:
sp = pd.read_csv('../input/sp-500-stocks/sp500_index.csv')
sp = sp.dropna()
sp["Date"] = pd.to_datetime(sp['Date'])
sp = sp[sp["Date"]>='2015-01-01']
sp.to_csv('sp.csv', index=False)

In [11]:
sp

,Date,S&P500
176,2015-01-02,2058.20
177,2015-01-05,2020.58
178,2015-01-06,2002.61
179,2015-01-07,2025.90
180,2015-01-08,2062.14
...,...,...
2512,2024-04-16,5051.41
2513,2024-04-17,5022.21
2514,2024-04-18,5011.12
2515,2024-04-19,4967.23
